In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# kaggle datasets download chilledwanker/loan-approval-prediction

In [3]:
from zipfile import ZipFile
with ZipFile('loan-approval-prediction.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall('../data')

In [4]:
import os
import pandas as pd

# Define the base path and the relative path
base_path = r"C:\Users\HP\Documents\Kaggle Projects\Loan Approval Prediction\data"
train_file_name = "playground-series-s4e10/train.csv"
test_file_name = "playground-series-s4e10/test.csv"
sub_file_name = "playground-series-s4e10/sample_submission.csv"
# Join paths
train_file_path = os.path.join(base_path, train_file_name)
test_file_path = os.path.join(base_path, test_file_name)
sub_file_path = os.path.join(base_path, sub_file_name)
orig_file_path = os.path.join(base_path, "credit_risk_dataset.csv")
# Load CSV
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)
sub_df = pd.read_csv(sub_file_path)

original_df = pd.read_csv(orig_file_path)




In [5]:
# import pandas as pd

# train_df = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
# test_df = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
# sub_df = pd.read_csv('/kaggle/input/playground-series-s4e10/sample_submission.csv')
# original_df = pd.read_csv('/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv')

In [6]:
train_df = train_df.drop(columns="id")
train_df = pd.concat([train_df, original_df], axis=0)


In [7]:
cat_cols = [col for col in train_df.select_dtypes(exclude=['int', 'float']).columns if col not in ('id', 'loan_status')]
num_cols = [col for col in train_df.select_dtypes(include=['int', 'float']).columns if col not in ('id', 'loan_status')]

cat_cols, num_cols

(['person_home_ownership',
  'loan_intent',
  'loan_grade',
  'cb_person_default_on_file'],
 ['person_age',
  'person_income',
  'person_emp_length',
  'loan_amnt',
  'loan_int_rate',
  'loan_percent_income',
  'cb_person_cred_hist_length'])

In [8]:
train_df.isna().sum()

person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
loan_status                      0
dtype: int64

In [ ]:
# na_df = train_df.isna().sum()
# na_cols  = [col for col in na_df.index if na_df[col] > 0]
# na_cols

: 

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import Ridge, Lasso
import copy

ridge = Ridge(alpha=1.0)
lasso = Lasso(alpha=1.0)

imputed_dfs = {}
for model in [ridge, lasso]:
    df = copy.deepcopy(train_df)
    df = df[num_cols]
    print(f'imputing using {model}')
    imputer = IterativeImputer(estimator=model, max_iter=1500, random_state=0, tol=1e-1)
    imputed_df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns, index=df.index )
    model_name = model.__class__.__name__
    imputed_dfs[model_name] = imputed_df



imputing using Ridge()


In [ ]:
ridge_imputed_df = imputed_dfs['Ridge']
train_df[num_cols] = ridge_imputed_df[num_cols]
train_df.isna().any()

In [ ]:
train_df['income_to_loan_ratio'] = train_df['person_income'] / train_df['loan_amnt']
train_df['age_emp_length']  = train_df['person_age'] * train_df['person_emp_length']


In [ ]:
cat_cols = [col for col in train_df.select_dtypes(exclude=['int', 'float']).columns if col not in ('id', 'loan_status')]
num_cols = [col for col in train_df.select_dtypes(include=['int', 'float']).columns if col not in ('id', 'loan_status')]

cat_cols, num_cols

In [ ]:
import matplotlib.pyplot as plt
import copy 
def box_plots_num_cols(df:pd.DataFrame, num_columns:list[str]):
    df = copy.deepcopy(df)
    base_width = 5
    base_height = 3
    columns = num_columns
    cols = round(len(columns) / 2)
    rows = len(columns) - cols
    fig_width = cols * base_width
    fig_height = rows * base_height
    fig, axes = plt.subplots(rows, cols, figsize=(fig_width, fig_height))
    axes = axes.flatten() if rows > 1 else [axes]
    for i, col in enumerate(columns):
        axes[i].boxplot(df[col])
        axes[i].set_title(col)
    for j in range(i+1, len(axes)):
        axes[j].axis('off')
    plt.tight_layout()

box_plots_num_cols(train_df, num_cols)

In [ ]:
from scipy import stats
import numpy as np
import pandas as pd

def remove_outliers(df: pd.DataFrame, cols: list[str], beta: float, cols_2: list[str] = None, beta2: float = None):
    # Avoid deep copy unless truly needed to optimize memory usage
    df_filtered = df.copy()
    beta_dict = {col: beta2 if cols_2 and col in cols_2 and beta2 is not None else (6.0 if col == 'person_income' else beta) for col in cols}

    for col in cols:
        data = df_filtered[col]
        
        if data.empty:
            print(f"Column {col} is empty, skipping...")
            continue
        
        # Use pandas quantile for faster percentile calculation
        iqr = stats.iqr(data)
        q1, q3 = data.quantile(0.25), data.quantile(0.75)
        beta_val = beta_dict[col]
        
        print(f'{col} - beta: {beta_val}')
        outlier_low = q1 - beta_val * iqr
        outlier_high = q3 + beta_val * iqr
        df_filtered = df_filtered[(data >= outlier_low) & (data <= outlier_high)]
        
    return df_filtered

# Run with specified parameters
df_clean = remove_outliers(train_df, num_cols, beta=1.5, cols_2=['loan_amnt'], beta2=3.0)


In [ ]:
def ins(indexx, clean_desc, orig_desc):
    # Fetch the row for the current index without extra conversions
    clean_values = clean_desc.loc[indexx].values
    orig_values = orig_desc.loc[indexx].values
    diff = orig_values - clean_values
    # Create DataFrame once for both sets of values
    res_df = pd.DataFrame({
        'orig': orig_values, 
        'clean': clean_values,
        'difference':diff
    }, index=num_cols)
    print(res_df.sort_values(by=['difference'], ascending=False, key= lambda x: abs(x)))

# Compute describe() once for both DataFrames
clean_desc = df_clean[num_cols].describe()
orig_desc = train_df[num_cols].describe()

# Loop through the index values
for index in ['count', 'mean', 'std', 'min', 'max']:
    
    print(f'\t\t\t{index.upper()}')
    print('*' * 50)
    ins(index, clean_desc, orig_desc)  # Pass precomputed descriptions
    print('-' * 50)
